# Risk Sensative Control

## Introdunction
Exisiting autonoumous driving controls assume that the users of the autonoumous driving system prefer the similar pattern, in practical scenarios, this is rarely the case. This paper introduces Risk Sensative Control (RSC), an inverse optimal control algorithm that is able to estimate the risk sensative driving features, and incorporate those features online into a receding-horizon cotroller. RSC solves the inverse optimal control problem by reducing subjective risk perseption from users feedback. RSC uses an Meta-learning algorithm to update the parameters of the cost function parameters, and  improve the controller online as more data is gathered from the users subjective risk feedback. The estimater take acounts of the uncertainty in the subjective risk analysis in driving features and surroudning vehicle locations, by uncertainty ellipse constraints. This approach is verified in realtime in five lane change both safe and risky scenarios in CARLA simulation environments.


## Related Work

### Subjective Risk Estimation

### Objective Function Estimation
Estimating objective functions from historical data is a well investigated problem known as Inverse Optimal Control
(IOC) or Inverse Reinforcement Learning (IRL). Typically, with the IRL approach, the objective function is linear in
terms of a given set of state features. The goal is to identify a parameter vector that weights these features so that
the behavior resulting from this estimated objective matches the observed behavior. 
Optimal control methods are optimizers, but in reality, we don’t really know what we should be optimizing. The question of how we can, instead, leverage our strong intuition of how we’d like the robot to behave to learn a cost function that embodies demonstrations of that behavior. So defines Inverse Optimal Control (IOC). Rather than computing an optimal policy given a cost function, IOC uncovers the cost function that best explains demonstrated optimal behavior. 

### Online Parameter Estimation
We review approaches proposed for online parameter estimation in this context. The Intelligent Driver Model (IDM), is a rule-based lane-following model balancing two objectives: reaching a desired speed and avoiding collision with
the preceding car. This model has few tunable parameters and returns the scalar acceleration of the vehicle along a predefined
path. Several works used online filtering techniques to estimate these IDM parameters.


### Data-Driven driving Control
There is a rich literature on applying data-driven approaches
to pedestrian or vehicle trajectory predictions. Such approaches usually require a large corpus of data and are trained
offline as a general model to suit multiple agents. On the other hand, we require a small amount of data and find parameters
online for a specific agent. Data-driven methods can predict a distribution over future trajectories e.g., offline inverse optimal
control with online goal inference; Conditional Variational Autoencoders (CVAE) or Generative Adversarial Networks (GAN). 

## Problem formulation

- Ego vehicle state: $$z=\begin{bmatrix}x\\ y\\ \phi\\ v\\ \omega\\  l \\ \dot{a} \\ \ddot{a}\end{bmatrix}$$

which represents $x, y$ position of trajectory, yaw angle, velocity, lateral acceleration, longitudinal acceleration, derivative of accel force, second order derivative of accel force, derivative of brake force, second order derivative of brake force.

- Ego vehicle control $$u = \begin{bmatrix}\delta\\a\end{bmatrix}$$

which represents steering angle, accel force, or brake force.
 

In [1]:
import math
import numpy as np
import scipy.interpolate

In [ ]:
# PARAMETER
L = 2.5  # wheel base
ds = 0.1  # course distanse
TARGET_SPEED = 30.0 / 3.6  # velocity [m/s]


In [3]:
class State:
    def __init__(self, x=0.0, y=0.0, yaw=0.0, v=0.0, omega=0.0, la=0.0, da=0.0, dda=0.0):
        self.x = x
        self.y = y
        self.yaw = yaw
        self.v = v
        self.omega = omega
        self.la = la
        self.da = da
        self.dda = dda
        

In [4]:
class Input:
    def __init__(self, delta=0, a=0):
        self.delta = 0
        self.a = 0
        

## Problem Formulation

## Problem Statement

Our lane change problem can be defined as the following figure. Red vehicle is our ego vehicle we is supposed to lane change to the next lane to pass the preceding vehicle. We focus on the discretized dynamic state-space form setting within length $N$ time horizon. We denote $z_t \in R^n$ the state and $u_t \in R^m$ the control input of the ego vehicle in $t \in \{1,...,N\}$ time steps. Ego vehicle's strategy is a control input sequence $U = [u_{t+1}, u_{t+2}, ...,u_{t+N}]$ and $u_{t+1}$ will be applied in each loop. We donote $z_t^k, k \in {1,...,6}$ to represent the state of surrounding vehicle in $k$ direction relative to ego vehicle in $t$ time step, and $u_t^k$ to represent the control input of surrounding vehicles. We assume the surrounding vehicle to drive in the same manner of ego vehicle but in the default parameter setting without lane changing.


<img src=./fig/scenario_1.png width="500px" >


## Features and Objectives


Since we use CARLA simulation environment to demenstrate our experiment, we list up the features information as following.



Here is the rotation definition in CARLA.

<img src=./fig/carla_rotation.jpg width="500px" >

In [1]:
import os
import glob
import sys
from carla import ColorConverter as cc

try:
    sys.path.append(glob.glob('../carla/dist/carla-*%d.%d-%s.egg' % (
        sys.version_info.major,
        sys.version_info.minor,
        'win-amd64' if os.name == 'nt' else 'linux-x86_64'))[0])
except IndexError:
    pass

In [2]:
class ModelPredictiveControl(object):
    def __init__(self, world, target_traj, start_in_autopilot):
        self._autopilot_enabled = start_in_autopilot # Keep autopilot
        self.target_traj = target_traj
        
        self._current_x          = 0
        self._current_y          = 0
        self._current_yaw        = 0
        self._current_speed      = 0
        self._desired_speed      = 0
        self._current_frame      = 0
        self._current_timestamp  = 0
        self._start_control_loop = False
        self._set_throttle       = 0
        self._set_brake          = 0
        self._set_steer          = 0
        self._waypoints          = waypoints
        self._conv_rad_to_steer  = 180.0 / 70.0 / np.pi
        self._pi                 = np.pi
        self._2pi                = 2.0 * np.pi
        
    def update_values(self, x, y, yaw, speed, timestamp, frame):
        self._current_x         = x
        self._current_y         = y
        self._current_yaw       = yaw
        self._current_speed     = speed
        self._current_timestamp = timestamp
        self._current_frame     = frame
        if self._current_frame:
            self._start_control_loop = True
    
    def parse_events(self, world, clock):
        if not self._autopilot_enabled:
            if isinstance(self._control, carla.VehicleControl):carla.Location(carla.Vector3D), 
                self._parse_vehicle_keys(pygame.key.get_pressed(), clock.get_time())
                self._parse_vehicle_wheel()
                self._control.reverse = self._control.gear < 0
            elif isinstance(self._control, carla.WalkerControl):
                self._parse_walker_keys(pygame.key.get_pressed(), clock.get_time())
            
            self._parse_naren_control(world)
            world.player.apply_control(self._control)
        
        
    def _parse_naren_control(self, world):
        [lc_x, lc_y, lc_yaw] = self.target_traj
        print('Here we can get mpc control in each time step\n')
        
        t = world.player.get_transform() # Return carla.Transform(carla.Vector3D), carla.Rotation(pitch(Y),yaw(Z), roll(X))[degree] 
        location = world.player.get_location #Return: carla.Location – meters
        v = world.player.get_velocity()  # Return: carla.Vector3D – m/s,v.x,v.y.v.z
        c = world.player.get_control()   # Return: carla.VehicleControl:throttle:[0,1], steer:[-1,1], brake:[0,1]
        a = world.player.get_acceleration() #Return: carla.Vector3D – m/s2
        w = world.player.get_angular_velocity() #Return: carla.Vector3D – deg/s
        
        
        

IndentationError: unexpected indent (<ipython-input-2-c39765cd9b89>, line 35)

In [1]:
def update(prev_state, u, dt, L):

    x_t = prev_state.x
    y_t = prev_state.y
    yaw_t = prev_state.yaw
    v_t = prev_state.v
    omega_t = prev_state.omega
    la_t = prev_state.la
    da_t = prev_state.da
    dda_t = prev_state.dda
    db_t = prev_state.db
    ddb_t = prev_state.ddb
    
    delta_t = delta
    if a>0 and b==0:
        a_t = a
        da_t_dot = a_t*dt
        dda_t_dot = da_t *dt
        db_t_dot = 0
        ddb_t_dot = 0
    elif a==0 and b>0:
        a_t = b
        da_t_dot = 0
        dda_t_dot = 0
        db_t_dot = a_t*dt
        ddb_t_dot = db_t*dt
    elif a==0 and b==0:
        a_t = 0
        da_t_dot = 0
        dda_t_dot = 0
        db_t_dot = 0
        ddb_t_dot = 0
    else:
        print('There is something wrong with the pedal force')
        
    x_dot = v_t *np.cos(yaw_t)
    y_dot = v_t*np.sin(yaw_t)
    v_t_dot = a_t
    yaw_t_dot = v_t*np.tan(delta_t)/L
    
    omega_t_dot = a_t*np.sin(yaw_t-delta_t)
    la_t_dot = a_t*np.cos(yaw_t-delta_t)
    
    
    state.x = x_t + x_dot*dt
    state.y = y_t + y_dot*dt
    state.yaw = pi_2_pi((yaw_t + yaw_t_dot)*dt)
    state.v = v_t + v_t_dot*dt
    state.omega = omega_t + omega_t_dot*dt
    state.la = la_t + la_t_dot*dt
    state.da = da_t + da_t_dot*dt
    state.db = db_t + db_t_dot*dt
    state.dda = dda_t + dda_t_dot*dt
    state.ddb = ddb_t + ddb_t_dot*dt

    return state


### Risk Sensative Motion Model


The vehicle dynamic is shown in this figure.
<img src=motion_model.png width="200px" >

We define our motion model as follows:
$$
\begin{eqnarray}
\dot{x} &=& v cos(\phi) \\
\dot{y} &=& v sin(\phi)\\
\dot{v} &=& a\\
\dot{\phi} &=& \frac{v}{L} tan(\delta)\\
\dot{\omega} &=&  a sin(\phi-\delta)\\
\dot{l}&=&  a cos(\phi-\delta)\\
\dot{a} &=& a dt\\
\ddot{a} &=& \dot{a} dt\\
\end{eqnarray}
$$
where $L$ is the length between the front and reer wheel.

It stems from executing the control strategies of all the surrouding vehicles on a joint dyanamical systems.
$$ z_{t+1} = A z_{t} + B u_t +C \\
\dot{z} = f(z,u) = A'z + B'u  
$$

ODE can be witten as
$$
\frac{\partial f}{\partial z} =
A'= 
\begin{bmatrix}
0&0&-v sin(\phi)&cos(\phi)&0&0&0&0\\
0&0& v cos(\phi)&sin(\phi)&0&0&0&0\\
0&0& 0&0&0&0&0&0\\
0&0& 0&tan{\delta}/L&0&0&0&0\\
0&0& a cos(\phi-\delta)&0&0&0&0&0\\
0&0& -a sin(\phi-\delta)&0&0&0&0&0\\
0&0& 0&0&0&0&0&0\\
0&0& 0&0&0&0&1&0
\end{bmatrix}
$$

$$
\frac{\partial f}{\partial u} =
B' = 
\begin{bmatrix}
0&0\\
0&0\\
0&1\\
\frac{v}{L cos^2(\delta)}&0\\
-a cos(\phi-\delta)&sin(\phi-\delta)\\
-a sin(\phi-\delta)&cos(\phi-\delta)\\
0&dt\\
0&0\\
\end{bmatrix}
$$

We can get a discrete-time mode with Forward Euler Discretization with sampling time $dt$.

$$z_{t+1} = z_k + f(z_t, u_t)dt$$

Using first degree Tayler expantion around $\bar{z}$ and $\bar{u}$
$$z_{t+1} = z_t + (f({\bar{z}, \bar{u}}) + A'z_t + B'u_t-A'\bar{z}-B'\bar{u})dt$$
$$z_{t+1} = (I+dtA')z_t + (dtB')u_t + (f(\bar{z},\bar{u})-A'\bar{z}-B'\bar{u}))dt$$

Therefore, we get
$$
A= 
\begin{bmatrix}
1&0&-v sin(\phi)&cos(\phi)&0&0&0&0\\
0&1& v cos(\phi)&sin(\phi)&0&0&0&0\\
0&0& 1&0&0&0&0&0\\
0&0& 0&1+tan{\delta}/L&0&0&0&0\\
0&0& a cos(\phi-\delta)&0&1&0&0&0\\
0&0& -a sin(\phi-\delta)&0&0&1&0&0\\
0&0& 0&0&0&&1&0\\
0&0& 0&0&0&0&1&1
\end{bmatrix}
$$

$$
B = 
\begin{bmatrix}
0&0\\
0&0\\
0&dt\\
dt\frac{v}{L*cos^2(\delta)}&0\\
-a*cos(\phi-\delta)dt&sin(\phi-\delta)dt\\
-a*sin(\phi-\delta)dt&cos(\phi-\delta)dt\\
0&dt\\
0&0\\
\end{bmatrix}
$$

$$
f(\bar{z},\bar{u})-A'\bar{z}-B'\bar{u}) = \\
\begin{bmatrix}
\bar{v} cos(\bar{\phi}) \\
\bar{v} sin(\bar{\phi})\\
\bar{a}\\
\frac{\bar{v}}{L} tan(\bar{\delta})\\
\bar{a}sin(\bar{\phi}-\bar{\delta})\\
\bar{a}cos(\bar{\phi}-\bar{\delta})\\
\bar{a}dt\\
\bar{\dot{a}}dt\\
\end{bmatrix}
-
\begin{bmatrix}
0&0&-v sin(\phi)&cos(\phi)&0&0&0&0\\
0&0& v cos(\phi)&sin(\phi)&0&0&0&0\\
0&0& 0&0&0&0&0&0\\
0&0& 0&tan{\delta}/L&0&0&0&0\\
0&0& a cos(\phi-\delta)&0&0&0&0&0\\
0&0& -a sin(\phi-\delta)&0&0&0&0&0\\
0&0& 0&0&0&0&0&0\\
0&0& 0&0&0&0&1&0
\end{bmatrix}
\begin{bmatrix}\bar{x}\\ \bar{y}\\ \bar{\phi}\\ \bar{v}\\ \bar{\omega}\\  \bar{l} \\ \bar{\dot{a}} \\ \bar{\ddot{a}}\end{bmatrix}-
\begin{bmatrix}
0&0\\
0&0\\
0&1\\
\frac{v}{L*cos^2(\delta)}&0\\
-a*cos(\phi-\delta)&sin(\phi-\delta)\\
-a*sin(\phi-\delta)&cos(\phi-\delta)\\
0&dt\\
0&0\\
\end{bmatrix}
\begin{bmatrix}\bar{\delta}\\\bar{a}\end{bmatrix}$$


$$=
\begin{bmatrix}
\bar{v} cos(\bar{\phi}) \\
\bar{v} sin(\bar{\phi})\\
\bar{a}\\
\frac{\bar{v}}{L} tan(\bar{\delta})\\
\bar{a}sin(\bar{\phi}-\bar{\delta})\\
\bar{a}cos(\bar{\phi}-\bar{\delta})\\
\bar{a}dt\\
\bar{\dot{a}}dt\\
\end{bmatrix} -
\begin{bmatrix}
-\bar{v}*sin(\bar{\phi})\bar{\phi} +\bar{v}cos(\bar{\phi})\\
 \bar{v}*cos(\bar{\phi})\bar{\phi} +\bar{v}sin(\bar{\phi})\\
0\\
tan({\bar\delta})\bar{v}/L\\
\bar{a}cos({\bar{\phi}-\bar{\delta}})\bar{\phi}\\
-\bar{a}sin({\bar{\phi}-\bar{\delta}})\bar{\phi} \\
0\\
0
\end{bmatrix}
-
\begin{bmatrix}
0\\
0\\
\bar{a}\\
\frac{\bar{v}}{L*cos^2(\bar{\delta})}\bar{\delta}\\
\bar{v}\bar{\delta}cos(\bar{\phi}-\bar{\delta})\bar{\delta} + sin(\bar{\phi}-\bar{\delta})\bar{a} \\
-\bar{v}\bar{\delta}sin(\bar{\phi}-\bar{\delta})\bar{\delta}+ cos(\bar{\phi}-\bar{\delta})\bar{a}\\
\bar{a}dt\\
0
\end{bmatrix}
=
\begin{bmatrix}
\bar{v}*sin(\bar{\phi})\bar{\phi} \\
-\bar{v}*cos(\bar{\phi})\bar{\phi}\\
0\\
-\frac{\bar{v}}{L*cos^2(\bar{\delta})}\bar{\delta}\\
\bar{\delta})-\bar{a}cos({\bar{\phi}-\bar{\delta}})\bar{\phi}-\bar{v}\bar{\delta}cos(\bar{\phi}-\bar{\delta})\bar{\delta}\\
-\bar{\delta})+\bar{a}sin({\bar{\phi}-\bar{\delta}})\bar{\phi}+\bar{v}\bar{\delta}sin(\bar{\phi}+ \bar{\delta})\bar{\delta} \\
0\\
\bar{\dot{a}}dt\\
\end{bmatrix}
$$

$$
C = \begin{bmatrix}
\bar{v}*sin(\bar{\phi})\bar{\phi} \\
-\bar{v}*cos(\bar{\phi})\bar{\phi}\\
0\\
-\frac{\bar{v}}{L*cos^2(\bar{\delta})}\bar{\delta}\\
-\bar{a}cos({\bar{\phi}-\bar{\delta}})\bar{\phi}-\bar{v}\bar{\delta}cos(\bar{\phi}-\bar{\delta})\bar{\delta}\\
\bar{a}sin({\bar{\phi}-\bar{\delta}})\bar{\phi}+\bar{v}\bar{\delta}sin(\bar{\phi}+ \bar{\delta})\bar{\delta} \\
0\\
\bar{\dot{a}}dt\\
\end{bmatrix}dt
$$


In a mylti-player driving situation, we assume the other agents are 'ideal' players. We further assume that ego vehicle and surrounding ego vehicles in a receding horizon loop.


### Objective Function Parameterization

$$
$$

$$ Contranits:
- Distance to surrounding vehicle


$$

In [ ]:
def main():
    argparser = argparse.ArgumentParser(
        description='CARLA Personalized Model Predictive Control Client')
    argparser.add_argument(
        '-v', '--verbose',
        action='store_true',
        dest='debug',
        help='print debug information')
    argparser.add_argument(
        '--host',
        metavar='H',
        default='127.0.0.1',
        help='IP of the host server (default: 127.0.0.1)')
    argparser.add_argument(
        '-p', '--port',
        metavar='P',
        default=2000,
        type=int,
        help='TCP port to listen to (default: 2000)')
    argparser.add_argument(
        '-a', '--autopilot',
        action='store_true',
        help='enable autopilot')
    argparser.add_argument(
        '--res',
        metavar='WIDTHxHEIGHT',
        default='1280x720',
        help='window resolution (default: 1280x720)')
    argparser.add_argument(
        '--filter',
        metavar='PATTERN',
        default='vehicle.*',
        help='actor filter (default: "vehicle.*")')
    args = argparser.parse_args()

    args.width, args.height = [int(x) for x in args.res.split('x')]

    log_level = logging.DEBUG if args.debug else logging.INFO
    logging.basicConfig(format='%(levelname)s: %(message)s', level=log_level)

    logging.info('listening to server %s:%s', args.host, args.port)

    print(__doc__)

    try:
        game_loop(args)

    except KeyboardInterrupt:
        print('\nCancelled by user. Bye!')

In [ ]:
if __name__ == '__main__':
    main()    